Import libraries

In [20]:
import pandas as pd
import numpy as np
import math
import re

In [21]:
DATA_FOLDER = 'data'
FOOD_FILE = 'food_data.csv'
COUNTRIES_FILE = 'countries_info.csv'

In [22]:
all_datas = pd.read_csv(DATA_FOLDER + "/" + FOOD_FILE, sep='\t')
countries_infos = pd.read_csv(DATA_FOLDER + "/" + COUNTRIES_FILE, sep=',')

/home/pirate/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0,3,5,24,25,26,28,35,36,41,49,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [23]:
for col in all_datas.columns.values:
    print("- ", col)

-  code
-  url
-  creator
-  created_t
-  created_datetime
-  last_modified_t
-  last_modified_datetime
-  product_name
-  generic_name
-  quantity
-  packaging
-  packaging_tags
-  brands
-  brands_tags
-  categories
-  categories_tags
-  categories_en
-  origins
-  origins_tags
-  manufacturing_places
-  manufacturing_places_tags
-  labels
-  labels_tags
-  labels_en
-  emb_codes
-  emb_codes_tags
-  first_packaging_code_geo
-  cities
-  cities_tags
-  purchase_places
-  stores
-  countries
-  countries_tags
-  countries_en
-  ingredients_text
-  allergens
-  allergens_en
-  traces
-  traces_tags
-  traces_en
-  serving_size
-  serving_quantity
-  no_nutriments
-  additives_n
-  additives
-  additives_tags
-  additives_en
-  ingredients_from_palm_oil_n
-  ingredients_from_palm_oil
-  ingredients_from_palm_oil_tags
-  ingredients_that_may_be_from_palm_oil_n
-  ingredients_that_may_be_from_palm_oil
-  ingredients_that_may_be_from_palm_oil_tags
-  nutrition_grade_uk
-  nutrition_grade_f

We can deduce the usefull colums for our project:
 - product_name
 - sugars_100g
 - fat_100g
 - nutrition_grade_fr
 - countries
 - countries_tags
 - origins
 - origins_tags
 - manufacturing_places
 - manufacturing_places_tags
 - glycemic-index_100g


In [24]:
countries_infos = countries_infos[['COUNTRY_ALPHA2_CODE', 'COUNTRY_NAME']]

In [28]:
usefull_datas = all_datas[['code',
                    'product_name',
                   'countries',
                   'countries_en',
                   'countries_tags',
                   'origins',
                   'origins_tags',
                   'manufacturing_places',
                   'first_packaging_code_geo',
                   'sugars_100g',
                   'fat_100g']]

check = []

for c in usefull_datas['countries_tags'].dropna():
    for t in c.split(','):
        t = t.strip().lower()
        yolo = re.search('\w\w:(.+)', t, re.IGNORECASE)
        
        if yolo:
            t = yolo.group(1)
        
        if t not in check:
            check.append(t)

countries_with_code = []

for c in check:
    is_present = False
    
    for index, c_i in countries_infos.iterrows():
        if c_i['COUNTRY_NAME'].lower() == c:
            countries_with_code.append((c, c_i['COUNTRY_ALPHA2_CODE']))
            is_present = True
            break
    
    if not is_present:
        countries_with_code.append((c, "???"))

375


[('60610-la-croix-st-ouen', '???'),
 ('afghanistan', 'AF'),
 ('aland-islands', '???'),
 ('albania', 'AL'),
 ('alemanha', 'DE'),
 ('alemania', 'DE'),
 ('algeria', 'DZ'),
 ('algerie', 'DZ'),
 ('all-over-the-world', '???'),
 ('allemagne', 'DE'),
 ('amerique-du-nord', '???'),
 ('amerique-du-sud', '???'),
 ('andorra', 'AD'),
 ('angola', 'AO'),
 ('anguilla', 'AI'),
 ('antigua-and-barbuda', '???'),
 ('antilles', '???'),
 ('apt', '???'),
 ('arabie-saoudite', '???'),
 ('argentina', 'AR'),
 ('armenia', 'AM'),
 ('aruba', 'AW'),
 ('auchan', '???'),
 ('australia', 'AU'),
 ('australie', 'AU'),
 ('austria', 'AT'),
 ('autriche', 'AT'),
 ('aveyron', '???'),
 ('azerbaijan', 'AZ'),
 ('bahrain', 'BH'),
 ('bangladesh', 'BD'),
 ('barbados', 'BB'),
 ('belarus', 'BY'),
 ('belgie', 'BE'),
 ('belgien', 'BE'),
 ('belgio', 'BE'),
 ('belgique', 'BE'),
 ('belgium', 'BE'),
 ('benelux', '???'),
 ('benin', 'BJ'),
 ('bolivia', 'BO'),
 ('bosnia-and-herzegovina', '???'),
 ('brands-completed', '???'),
 ('brazil', 'BR'),
 

We select only the countries with which we have informations.

In [33]:
count = 0
proper_countries = []

for c in countries_with_code:
    if c[1] != "???":
        proper_countries.append(c)
        
print(proper_countries, len(proper_countries))

[('afghanistan', 'AF'), ('albania', 'AL'), ('alemanha', 'DE'), ('alemania', 'DE'), ('algeria', 'DZ'), ('algerie', 'DZ'), ('allemagne', 'DE'), ('andorra', 'AD'), ('angola', 'AO'), ('anguilla', 'AI'), ('argentina', 'AR'), ('armenia', 'AM'), ('aruba', 'AW'), ('australia', 'AU'), ('australie', 'AU'), ('austria', 'AT'), ('autriche', 'AT'), ('azerbaijan', 'AZ'), ('bahrain', 'BH'), ('bangladesh', 'BD'), ('barbados', 'BB'), ('belarus', 'BY'), ('belgie', 'BE'), ('belgien', 'BE'), ('belgio', 'BE'), ('belgique', 'BE'), ('belgium', 'BE'), ('benin', 'BJ'), ('bolivia', 'BO'), ('brazil', 'BR'), ('brunei', 'BN'), ('bulgaria', 'BG'), ('burundi', 'BI'), ('cambodia', 'KH'), ('cameroon', 'CM'), ('cameroun', 'CM'), ('canada', 'CA'), ('chad', 'TD'), ('chile', 'CL'), ('china', 'CN'), ('chine', 'CN'), ('colombia', 'CO'), ('croatia', 'HR'), ('cuba', 'CU'), ('curacao', 'CW'), ('cyprus', 'CY'), ('danemarca', 'DK'), ('danemark', 'DK'), ('denmark', 'DK'), ('deutschland', 'DE'), ('dinamarca', 'DK'), ('djibouti', 'D

The objective is to calculate the mean of sugars inside food for european countries. Some countries_tags are weirdly format we have to process the data a little bit to be able to present a relevant result.

In [43]:
nb_by_countries = usefull_datas.groupby(['countries_tags']).size().reset_index(name='counts')

nb_by_countries = nb_by_countries.sort_values(by=['counts'], ascending=False)
#nb_by_countries

test = usefull_datas[np.isfinite(usefull_datas['sugars_100g'])]
test = test[['countries_tags', 'sugars_100g']].groupby(['countries_tags']).mean()
test = test.sort_values(by=['sugars_100g'], ascending=False)
test = test[test['sugars_100g'] != 0]
test

,sugars_100g
countries_tags,
"en:canada,en:french-polynesia",33300.000000
en:paraguay,3140.000000
en:peru,694.437143
en:india,521.337922
en:canada,371.963023
en:taiwan,291.889189
en:colombia,165.420500
en:greece,113.856251
"en:france,en:french-guiana,en:reunion",100.000000


In [26]:
sugar = usefull_datas[np.isfinite(usefull_datas['sugars_100g'])]

print("Nmber of product with informations relative to sugar : ", sugar.size)

sugar = sugar.sort_values(by=['sugars_100g'], ascending=False)[['product_name', 'sugars_100g']]
sugar

Nmber of product with informations relative to sugar :  5115933


,product_name,sugars_100g
116832,Confiture 4 Fruits,86700.00
339375,Sirop Tropical Carrefour,68000.00
76987,confiture à la fraise,60000.00
674787,bourbon biscuits,37500.00
73704,vinaigre balsamique figue,33300.00
185246,Cerises à tartiner,33300.00
142629,ketchup maison,20000.00
624112,pate concentrée de tomates,16700.00
508042,Frozen Watermelon,13000.00
586338,Jugo de Naranja sin pulpa,12400.00


In [27]:
fat = usefull_datas[np.isfinite(usefull_datas['fat_100g'])]

print("Nmber of product with informations relative to fat : ", fat.size)

fat = fat.sort_values(by=['fat_100g'], ascending=False)[['product_name', 'fat_100g']]
fat

Nmber of product with informations relative to fat :  5210478


,product_name,fat_100g
76621,Mayonnaise,66700.0
674787,bourbon biscuits,20000.0
675261,Spicy (Masala) Noddles Khakhara,18400.0
513755,Havermout,10000.0
71851,Québon,6670.0
641910,Flocs de civada,6500.0
610286,Leche Entera,3120.0
675241,Pillsbury cookie cake - choco nut butter,2700.0
497593,Kinder Überraschung Maxi,2413.0
544217,Παξιμάδια Κριθαρένια,2000.0
